In [1]:
# Import packages

import pandas as pd 
import numpy as np 
import os

pd.set_option("display.max_columns", None)

C:\Users\imacd_0odruq3\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
gambling_df = pd.read_csv('../data/truncated_cleaned_betting_data_closing_lines.csv')
gambling_df = gambling_df.drop(columns=['Unnamed: 0'])

In [3]:
# Move these to helper functions so it can be used later, once confirmed that it works

def calculate_ml_return(line, wager, outcome='win', return_includes_wager=False):
    if return_includes_wager:
        add_on = wager
    else:
        add_on = 0
        
    if outcome == 'win':
        if line > 0:
            return np.round(((line/100)*wager + add_on), 2)
        else:
            return np.round(((100/abs(line))*wager + add_on), 2)
        
    elif outcome == 'push':
        return add_on
    
    elif outcome == 'loss':
        return np.round(-wager, 2)
    
    else:
        raise ValueError("""Outcome must be one of: win, loss, push""")
        
def calculate_spread_return(line=-110, wager=None, outcome='win', return_includes_wager=False):
    if return_includes_wager:
        add_on = wager
    else:
        add_on = 0
        
    if outcome == 'win':
        if line > 0:
            return np.round(((line/100)*wager + add_on), 2)
        else:
            return np.round(((100/abs(line))*wager + add_on), 2)
        
    elif outcome == 'push':
        return add_on
    
    elif outcome == 'loss':
        return np.round(-wager, 2)
    
    else:
        raise ValueError("""Outcome must be one of: win, loss, push""")
        
def calculate_ou_return(line=-110, wager=None, outcome='win', return_includes_wager=False):
    if return_includes_wager:
        add_on = wager
    else:
        add_on = 0
        
    if outcome == 'win':
        if line > 0:
            return np.round(((line/100)*wager + add_on), 2)
        else:
            return np.round(((100/abs(line))*wager + add_on), 2)
        
    elif outcome == 'push':
        return add_on
    
    elif outcome == 'loss':
        return np.round(-wager, 2)
    
    else:
        raise ValueError("""Outcome must be one of: win, loss, push""")

In [4]:
# Move these to helper functions so it can be used later, once confirmed that it works

def get_favorite_params(game):
    if game.home_moneyline <= game.away_moneyline:
        return 'home'
    else:
        return 'away'
    
def get_underdog_params(game):
    if game.home_moneyline <= game.away_moneyline:
        return 'away'
    else:
        return 'home'

def get_ml_bet_params(game, bet):
    
    home_or_away = None
    outcome = None
    
    if bet == 'favorite':
        home_or_away = get_favorite_params(game)
    
    elif bet == 'underdog':
        home_or_away = get_underdog_params(game)
        
    elif bet == 'home':
        home_or_away = 'home'
    
    elif bet == 'away':
        home_or_away = 'away'
        
    if home_or_away == 'home' and game.game_winner == 'home':
        outcome = 'win'
    
    elif home_or_away == 'away' and game.game_winner == 'away':
        outcome = 'win'
        
    elif game.game_winner == 'push':
        outcome = 'push'
        
    else:
        outcome = 'loss'
        
    return home_or_away, outcome

def get_favorite_params_spread(game):
    if game.home_spread <= game.away_spread:
        return 'home'
    else:
        return 'away'
    
def get_underdog_params_spread(game):
    if game.home_spread <= game.away_spread:
        return 'away'
    else:
        return 'home'

def get_spread_bet_params(game, bet):
    
    home_or_away = None
    outcome = None
    
    if bet == 'spread_favorite':
        home_or_away = get_favorite_params_spread(game)
    
    elif bet == 'spread_underdog':
        home_or_away = get_underdog_params_spread(game)
        
    elif bet == 'spread_home':
        home_or_away = 'home'
    
    elif bet == 'spread_away':
        home_or_away = 'away'
        
    if home_or_away == 'home' and game.spread_cover_result == 'home':
        outcome = 'win'
    
    elif home_or_away == 'away' and game.spread_cover_result == 'away':
        outcome = 'win'
        
    elif game.spread_cover_result == 'push':
        outcome = 'push'
        
    else:
        outcome = 'loss'
        
    return home_or_away, outcome

def get_ou_bet_params(game, bet):
    
    outcome = None
    
    if bet == game.over_under_result:
        outcome = 'win'
        
    elif game.over_under_result == 'push':
        outcome = 'push'
        
    else:
        outcome = 'loss'
        
    return outcome 

def get_wager_returns(game, bet, base_wager=100):
    
    list_valid_bets = {'favorite', 'underdog', 'home', 'away', 'spread_favorite', 'spread_underdog',
                      'spread_home', 'spread_away', 'over', 'under'}
    
    if bet not in list_valid_bets:
        raise ValueError("""Invalid bet. Bet must be one of: favorite, underdog, home, away, spread_favorite,
        spread_underdog, spread_home, spread_away, over, under""")
        
    elif bet in {'favorite', 'underdog', 'home', 'away'}:
        home_or_away, outcome = get_ml_bet_params(game, bet)
        line = game['{}_moneyline'.format(home_or_away)]
        return calculate_ml_return(line, base_wager, outcome, False)
    
    elif bet in {'spread_favorite', 'spread_underdog', 'spread_home', 'spread_away'}:
        home_or_away, outcome = get_spread_bet_params(game, bet)
        line = -110
        return calculate_spread_return(line, base_wager, outcome, False)
    
    elif bet in {'over', 'under'}:
        outcome = get_ou_bet_params(game, bet)
        line = -110
        return calculate_ou_return(line, base_wager, outcome, False)

In [5]:
# Add columns for returns for all different bet types, with base wager of 100 dollars

base_wager = 100

gambling_df['favorite_return'] = gambling_df.apply(lambda x: get_wager_returns(x, 'favorite', base_wager), axis=1)
gambling_df['underdog_return'] = gambling_df.apply(lambda x: get_wager_returns(x, 'underdog', base_wager), axis=1)
gambling_df['home_return'] = gambling_df.apply(lambda x: get_wager_returns(x, 'home', base_wager), axis=1)
gambling_df['away_return'] = gambling_df.apply(lambda x: get_wager_returns(x, 'away', base_wager), axis=1)
gambling_df['favorite_spread_return'] = gambling_df.apply(lambda x: get_wager_returns(x, 'spread_favorite', base_wager), axis=1)
gambling_df['underdog_spread_return'] = gambling_df.apply(lambda x: get_wager_returns(x, 'spread_underdog', base_wager), axis=1)
gambling_df['home_spread_return'] = gambling_df.apply(lambda x: get_wager_returns(x, 'spread_home', base_wager), axis=1)
gambling_df['away_spread_return'] = gambling_df.apply(lambda x: get_wager_returns(x, 'spread_away', base_wager), axis=1)
gambling_df['over_return'] = gambling_df.apply(lambda x: get_wager_returns(x, 'over', base_wager), axis=1)
gambling_df['under_return'] = gambling_df.apply(lambda x: get_wager_returns(x, 'under', base_wager), axis=1)

In [6]:
# NOTE: Need to double check all these values and make sure they are correct

gambling_df.head()

,date,home_team,away_team,home_score,away_score,total_score_actual,home_moneyline,away_moneyline,home_spread,away_spread,total_score_line,over_under_result,home_team_actual_line,away_team_actual_line,spread_cover_result,home_implied_prob,away_implied_prob,game_winner,favorite_return,underdog_return,home_return,away_return,favorite_spread_return,underdog_spread_return,home_spread_return,away_spread_return,over_return,under_return
0,2023-02-12,Philadelphia Eagles,Kansas City Chiefs,35,38,73,-120.0,100.0,-1.0,1.0,51.5,over,3,-3,away,0.545455,0.500000,away,-100.00,100.0,-100.00,100.0,-100.00,90.91,-100.00,90.91,90.91,-100.00
1,2023-01-29,Kansas City Chiefs,Cincinnati Bengals,23,20,43,-130.0,110.0,-2.0,2.0,48.5,under,-3,3,home,0.565217,0.476190,home,76.92,-100.0,76.92,-100.0,90.91,-100.00,90.91,-100.00,-100.00,90.91
2,2023-01-29,Philadelphia Eagles,San Francisco 49ers,31,7,38,-155.0,135.0,-3.0,3.0,45.0,under,-24,24,home,0.607843,0.425532,home,64.52,-100.0,64.52,-100.0,90.91,-100.00,90.91,-100.00,-100.00,90.91
3,2023-01-22,San Francisco 49ers,Dallas Cowboys,19,12,31,-200.0,170.0,-3.5,3.5,46.5,under,-7,7,home,0.666667,0.370370,home,50.00,-100.0,50.00,-100.0,90.91,-100.00,90.91,-100.00,-100.00,90.91
4,2023-01-22,Buffalo Bills,Cincinnati Bengals,10,27,37,-260.0,215.0,-6.0,6.0,48.5,under,17,-17,away,0.722222,0.317460,away,-100.00,215.0,-100.00,215.0,-100.00,90.91,-100.00,90.91,-100.00,90.91


In [7]:
gambling_df.to_csv('../data/betting_data_cleaned_with_returns.csv')